# Import Libraries

In [1]:
# Import libraries
import numpy as np # any math operations
import pandas as pd # data wrangling and manipulation
import matplotlib.pyplot as plt # visuals
import seaborn as sns # visuals

import re # for regex

# My Functions

In [2]:
# MIGHT REMOVE LATER!!!!

#decribe dataframe
def describe_df(dataframe, categorical=False):
    if not dataframe.empty:
        if categorical == False:
            return dataframe.describe()
        else:
            return dataframe.describe(include='O')
    else:
        return "Your dataframe does not exist! Check your dataframe variable."


#check nulls
def check_nulls(dataframe):
    num_nulls = dataframe.isnull().sum()
    percent_nulls = (num_nulls / len(dataframe) * 100).round(2)
    if not dataframe.empty:
        if num_nulls.sum() > 0:
            print(f"Here are the number of nulls per column: \n{num_nulls}")
            print(f"Here are the percent of nulls per column: \n{percent_nulls}")
        else:
            return "There are no nulls in your data."
    else:
        return "Your dataframe does not exist! Check your dataframe variable."

#check value counts
def unique(dataframe):
    if not dataframe.empty:
        return f"Here are the number of unique values per columns:", dataframe.nunique()


#check for dups
def check_dups(dataframe):
    num_dups = dataframe.duplicated().sum()
    if not dataframe.empty:
        return f"There are {num_dups} duplicates in your dataframe"
    else:
        return "Your dataframe does not exist! Check your dataframe variable."
    
#drop dups
def drop_dups(dataframe):
    dups = check_dups(dataframe)
    if dups == 0:
        return dataframe
    else:
        return dataframe.drop_duplicates()


#Visuals 

def make_hist(dataframe, columns, num_rows, num_columns, fig_size):
    if not dataframe.empty:
       fig, axs = plt.subplots(num_rows, num_columns, figsize=fig_size)
       for i in range(num_rows):
           for j in range(num_columns):
               axs[i,j].hist(dataframe[columns[i * num_columns + j]])
               axs[i,j].set_title(columns[i * num_columns + j])
    else:
        return "Your dataframe is empty. Check your dataframe variable"

#make boxplots
def make_boxplot(dataframe, columns, num_rows, num_columns, fig_size):
    if not dataframe.empty:
       fig, axs = plt.subplots(num_rows, num_columns, figsize=fig_size)
       for i in range(num_rows):
           for j in range(num_columns):
               axs[i,j].boxplot(dataframe[columns[i * num_columns + j]])
               axs[i,j].set_title(columns[i * num_columns + j])
    else:
        return "Your dataframe is empty. Check your dataframe variable"

#make pairplot
def make_pairplot(dataframe):
    return sns.pairplot(dataframe)

#make heatmap
def make_heatmap(dataframe):
    correlation = dataframe.corr(numeric_only=True)
    sns.heatmap(correlation, cmap='coolwarm', annot=True)
    return plt.show()

#make scatter plot
def make_scatter(dataframe, x_column, y_column, title: str, x_label: str, y_label: str):
    plt.scatter(x=dataframe[x_column], y=dataframe[y_column])
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    return plt.show()

# Data Cleaning and Formatting

In [50]:
#load data - ignore errors and load as string; it was easier

raw_data = pd.read_csv('38050-0001-Data.tsv', sep='\t', dtype=str, encoding_errors='ignore')

#make copy to work on
data = raw_data.copy()

In [3]:
#get random samples to see format
data.sample(10)

,CASEID,NAME,PID,CATEGORY,CATEGORY_ID,SUBCATEGORY,SUBCATEGORY_ID,PROJECT_PAGE_LOCATION_NAME,PROJECT_PAGE_LOCATION_STATE,PROJECT_PAGE_LOCATION_COUNTY,...,LAUNCHED_DATE,DEADLINE_DATE,PROJECT_CURRENCY,GOAL_IN_ORIGINAL_CURRENCY,PLEDGED_IN_ORIGINAL_CURRENCY,GOAL_IN_USD,PLEDGED_IN_USD,BACKERS_COUNT,STATE,URL_NAME
392623,392624,MASKED BY ICPSR,1007612426,Music,14,Music,14,Minneapolis,MN,Hennepin,...,2/2/2012,2/23/2012,USD,"1,500","2,025","$1,500","$2,025",58,successful,MASKED BY ICPSR
382394,382395,MASKED BY ICPSR,1404727536,Art,1,Installations,288,Black Rock City,NV,Washoe,...,4/17/2015,5/17/2015,USD,"2,000",147,"$2,000",$147,5,failed,MASKED BY ICPSR
184319,184320,MASKED BY ICPSR,1377484490,Technology,16,Apps,332,Washington,DC,District of Columbia,...,4/3/2019,5/8/2019,USD,"10,000","1,032","$10,000","$1,032",16,failed,MASKED BY ICPSR
87421,87422,MASKED BY ICPSR,388213468,Film & Video,11,Drama,293,Fort Wayne,IN,Allen,...,3/15/2015,4/14/2015,USD,"25,000",0,"$25,000",$0,0,failed,MASKED BY ICPSR
226853,226854,MASKED BY ICPSR,1615842379,Music,14,Pop,42,,,,...,6/1/2010,7/5/2010,USD,"7,000","1,466","$7,000","$1,466",35,failed,MASKED BY ICPSR
369958,369959,MASKED BY ICPSR,1911446919,Fashion,9,Apparel,263,Easley,SC,Pickens,...,1/5/2016,2/4/2016,USD,"2,000",56,"$2,000",$56,2,failed,MASKED BY ICPSR
463391,463392,MASKED BY ICPSR,331620360,Design,7,Product Design,28,Richmond,TX,Fort Bend,...,8/6/2014,8/30/2014,USD,500,630,$500,$630,8,successful,MASKED BY ICPSR
159570,159571,MASKED BY ICPSR,387676777,Fashion,9,Footwear,266,Singapore,Central Singapore,,...,11/25/2020,12/30/2020,SGD,"11,800",7,"$8,818",$5,2,canceled,MASKED BY ICPSR
81554,81555,MASKED BY ICPSR,686157190,Food,10,Food,10,Pasadena,CA,Los Angeles,...,5/18/2015,6/22/2015,USD,"28,000","28,805","$28,000","$28,806",106,successful,MASKED BY ICPSR
139573,139574,MASKED BY ICPSR,774319074,Design,7,Product Design,28,Milan,Lombardy,Milan,...,2/18/2017,3/23/2017,EUR,"15,000","24,173","$16,184","$26,081",528,successful,MASKED BY ICPSR


In [51]:
#check for duplicates before proceeding and always do it when you have the id column; duplicates will arise otherwise

data.duplicated().sum() # Use sum to get number instead of boolean

0

In [52]:
# drop Name and URL name since it brings no value

data = data.drop(['NAME', 'URL_NAME'], axis=1)

In [53]:
#reformat feature names
#let's format and rename the columns so it doesn't look like we are yelling

data.columns = data.columns.str.capitalize()


# There is a more simple way to do the renaming part, but when I'm changing feature names I like to have a key so I can refer back to it if I need to.
# Some of the column names are too long
new_features = {'Caseid': 'Case_ID', 
                'Project_page_location_name': 'Project_Country',
                'Project_page_location_state': 'Project_State',
                'Project_page_location_county': 'City',
                'Launched_date': 'Launched',
                'Deadline_date': 'Deadline',
                'Project_currency': 'Project_Currency',
                'Goal_in_original_currency': 'Goal',
                'Pledged_in_original_currency': 'Pledged',
                'Goal_in_usd': 'Goal (USD)',
                'Pledged_in_usd': 'Pledged (USD)',
                'Backers_count': 'Backers (#)',
                'State': 'Status'}



#change column names
data.rename(columns=new_features, inplace=True)

#check changes were made
data.sample(10)

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Project_State,City,Uid,Launched,Deadline,Project_Currency,Goal,Pledged,Goal (USD),Pledged (USD),Backers (#),Status
363673,363674,1935247215,Film & Video,11,Shorts,32,Vilnius,Vilnius County,Vilnius City Municipality,1156037670,2/8/2016,3/9/2016,EUR,"2,200",945,"$2,414","$1,037",25,failed
50581,50582,617041185,Food,10,Farms,309,Queretaro,Queretaro de Arteaga,Queretaro,827848580,7/26/2019,9/24/2019,MXN,"50,000",628,"$2,563",$32,4,failed
139098,139099,1758934583,Film & Video,11,Documentary,30,Oakland,CA,Alameda,311983644,12/3/2012,1/2/2013,USD,"15,000",586,"$15,000",$586,16,failed
398016,398017,1475141764,Film & Video,11,Webseries,33,Denver,CO,Denver,489818798,4/6/2012,5/6/2012,USD,"1,500",10,"$1,500",$10,1,failed
495759,495760,907699039,Art,1,Illustration,22,Sherman Oaks,CA,Los Angeles,2125561390,3/17/2015,3/27/2015,USD,150,190,$150,$190,32,successful
243806,243807,1960525433,Photography,15,Photography,15,Flint,MI,Genesee,2038990664,9/19/2013,10/20/2013,USD,"6,000","6,829","$6,000","$6,830",155,successful
498088,498089,1375673737,Publishing,18,Children's Books,46,Swan River,MB,Manitoba,144442609,9/26/2019,11/24/2019,CAD,100,248,$75,$187,8,successful
264961,264962,1205215881,Fashion,9,Fashion,9,Atlanta,GA,Fulton,308096350,1/22/2014,2/21/2014,USD,"5,000",869,"$5,000",$869,11,failed
42357,42358,1082223657,Film & Video,11,Film & Video,11,Mohnton,PA,Berks,928998307,2/4/2013,3/26/2013,USD,"50,000","50,921","$50,000","$50,922",501,successful
244248,244249,2114691232,Art,1,Performance Art,24,Montgomery,AL,Montgomery,1664941407,6/13/2012,7/18/2012,USD,"6,000",55,"$6,000",$55,2,failed


In [24]:
# check for nulls
data.isna().sum()

Case_ID             0
Pid                 0
Category            0
Category_id         0
Subcategory         0
Subcategory_id      0
Project_Country     0
Project_State       0
City                0
Uid                 0
Launched            0
Deadline            0
Project_Currency    0
Goal                0
Pledged             0
Goal (USD)          0
Pledged (USD)       0
Backers (#)         0
Status              0
dtype: int64

Using .isna() suggests there are no nulls, but in a tsv, empty values are recorded as a tab/space and not nan. So, be aware of 

In [54]:
#check for empty strings first given its a tsv
empty_strings = data.apply(lambda x: x == ' ') # apply lambda function to check for spaces and create mask
count_empty = empty_strings.sum() # get total instances in each feature
print(count_empty)

Case_ID                 0
Pid                     0
Category                0
Category_id             0
Subcategory             0
Subcategory_id          0
Project_Country      1873
Project_State        2014
City                64082
Uid                     0
Launched                0
Deadline                0
Project_Currency        0
Goal                    0
Pledged                 0
Goal (USD)              0
Pledged (USD)           0
Backers (#)         10148
Status                  0
dtype: int64


Empty strings in Project_Country, Project_State, City, and Backers (#)
- Can use Project_Currency to fill in empty countries,
- City can probably be dropped; not worth filling in
- Backers will be interesting - could drop (last resort), impute will depend on whether the campaign is successful or not

First let's deal with numeric columns - features that should be numeric, but currently strings

In [55]:
# declare numeric features, remember all features were imported as a string
numeric_columns = ['Goal', 'Pledged', 'Goal (USD)', 'Pledged (USD)', 'Backers (#)']

#replace characters
def replace_characters_empty(string):
    new_string = re.sub(r'[\$, ]', '', string)
    return new_string

#insert nan
def insert_nan(string):
    new_string = string.replace('', np.nan)
    return new_string

#change data types
def change_data_type(dataframe, columns, new_type):
   for column in columns:
       dataframe[column] = dataframe[column].astype(new_type)
   return dataframe.info()


In [56]:
#replace some regular expressions in numeric columns with empty string
for column in numeric_columns:
    data[column] = data[column].apply(replace_characters_empty)

#check work
data.head() 

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Project_State,City,Uid,Launched,Deadline,Project_Currency,Goal,Pledged,Goal (USD),Pledged (USD),Backers (#),Status
0,1,2137925650,Film & Video,11,Science Fiction,301,London,England,Greater London,1076478145,8/11/2016,10/10/2016,USD,100000000,0,100000000,0,0,canceled
1,2,1501531085,Film & Video,11,Fantasy,296,Los Angeles,CA,Los Angeles,224946798,12/19/2019,2/14/2020,USD,100000000,85,100000000,85,4,canceled
2,3,953415668,Technology,16,Software,51,Mexico,Baja California,Tijuana,1772203542,3/1/2017,3/22/2017,MXN,100000000,10,5219374,1,1,failed
3,4,1371386304,Publishing,18,Publishing,18,Columbus,OH,Franklin,1373465389,6/4/2018,7/5/2018,USD,100000000,1,100000000,1,1,canceled
4,5,1720842777,Art,1,Illustration,22,Toronto,ON,Toronto,1455666383,5/1/2015,6/30/2015,CAD,100000000,0,80610122,0,0,failed


In [57]:
# check for spaces

empty_strings = data.apply(lambda x: x == ' ') # apply lambda function to check for spaces
count_empty = empty_strings.sum() # get total instances in each feature
print(count_empty)

Case_ID                 0
Pid                     0
Category                0
Category_id             0
Subcategory             0
Subcategory_id          0
Project_Country      1873
Project_State        2014
City                64082
Uid                     0
Launched                0
Deadline                0
Project_Currency        0
Goal                    0
Pledged                 0
Goal (USD)              0
Pledged (USD)           0
Backers (#)             0
Status                  0
dtype: int64


In [58]:
# check for empty strings
empty_strings = data.apply(lambda x: x == '') # apply lambda function to check for spaces
count_empty = empty_strings.sum() # get total instances in each feature
print(count_empty)

Case_ID                 0
Pid                     0
Category                0
Category_id             0
Subcategory             0
Subcategory_id          0
Project_Country         0
Project_State           0
City                    0
Uid                     0
Launched                0
Deadline                0
Project_Currency        0
Goal                    0
Pledged                 0
Goal (USD)              0
Pledged (USD)           0
Backers (#)         10148
Status                  0
dtype: int64


Okay, need to replace backers with nan and will impute later

In [63]:
# replace empty string in Backers (#) with 
for column in numeric_columns:
    if column == 'Backers (#)':
        data[column] = data[column].replace('', np.nan) # replace empty string with np.nan
        data[column] = data[column].astype(pd.Int64Dtype()) # convert to int
    else:       
        data[column] = data[column].astype(int) # convert other numeric features to ints


# check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506199 entries, 0 to 506198
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Case_ID           506199 non-null  object
 1   Pid               506199 non-null  object
 2   Category          506199 non-null  object
 3   Category_id       506199 non-null  object
 4   Subcategory       506199 non-null  object
 5   Subcategory_id    506199 non-null  object
 6   Project_Country   506199 non-null  object
 7   Project_State     506199 non-null  object
 8   City              506199 non-null  object
 9   Uid               506199 non-null  object
 10  Launched          506199 non-null  object
 11  Deadline          506199 non-null  object
 12  Project_Currency  506199 non-null  object
 13  Goal              506199 non-null  int64 
 14  Pledged           506199 non-null  int64 
 15  Goal (USD)        506199 non-null  int64 
 16  Pledged (USD)     506199 non-null  int

*Will fill in nulls later*

In [64]:
#change Status values to title case

data['Status'] = data['Status'].str.title()

#change Launched and Deadline to datetime

data['Launched'] = pd.to_datetime(data['Launched'])
data['Deadline'] = pd.to_datetime(data['Deadline'])

# check
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506199 entries, 0 to 506198
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Case_ID           506199 non-null  object        
 1   Pid               506199 non-null  object        
 2   Category          506199 non-null  object        
 3   Category_id       506199 non-null  object        
 4   Subcategory       506199 non-null  object        
 5   Subcategory_id    506199 non-null  object        
 6   Project_Country   506199 non-null  object        
 7   Project_State     506199 non-null  object        
 8   City              506199 non-null  object        
 9   Uid               506199 non-null  object        
 10  Launched          506199 non-null  datetime64[ns]
 11  Deadline          506199 non-null  datetime64[ns]
 12  Project_Currency  506199 non-null  object        
 13  Goal              506199 non-null  int64         
 14  Pled

In [15]:
# check
data.head()

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Project_State,City,Uid,Launched,Deadline,Project_Currency,Goal,Pledged,Goal (USD),Pledged (USD),Backers (#),Status
0,1,2137925650,Film & Video,11,Science Fiction,301,London,England,Greater London,1076478145,8/11/2016,10/10/2016,USD,100000000,0,100000000,0,0,Canceled
1,2,1501531085,Film & Video,11,Fantasy,296,Los Angeles,CA,Los Angeles,224946798,12/19/2019,2/14/2020,USD,100000000,85,100000000,85,4,Canceled
2,3,953415668,Technology,16,Software,51,Mexico,Baja California,Tijuana,1772203542,3/1/2017,3/22/2017,MXN,100000000,10,5219374,1,1,Failed
3,4,1371386304,Publishing,18,Publishing,18,Columbus,OH,Franklin,1373465389,6/4/2018,7/5/2018,USD,100000000,1,100000000,1,1,Canceled
4,5,1720842777,Art,1,Illustration,22,Toronto,ON,Toronto,1455666383,5/1/2015,6/30/2015,CAD,100000000,0,80610122,0,0,Failed


Plan:
- use Project_Currency to fill in Project_Country
- drop Project_State and Project_City as it will be too much work to accurately fill them in

In [65]:
# make dictionary using project currency

currency_codes = {'USD': 'United States of America',
                  'EUR': 'European Union',
                  'GBP': 'United Kingdom',
                  'CAD': 'Canada', 
                  'AUD': 'Australia',
                  'MXN': 'Mexico',
                  'SEK': 'Sweden',
                  'HKD': 'Hong Kong',
                  'NZD': 'New Zealand',
                  'DKK': 'Denmark',
                  'SGD': 'Singapore',
                  'CHF': 'Switzerland',
                  'NOK': 'Norway',
                  'JPY': 'Japan',
                  'PLN': 'Poland'}

# replace Project_Country values with currency code dictionary
data['Project_Country'] = data['Project_Currency'].map(currency_codes)



In [66]:
# Drop project_state, city, Goal and pledged - these are redundant features. It is better to focus on USD numbers as Kickstarter is an American company and it provides consistency
data = data.drop(['Project_State', 'City', 'Goal', 'Pledged'], axis=1)

In [67]:
# check work
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506199 entries, 0 to 506198
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Case_ID           506199 non-null  object        
 1   Pid               506199 non-null  object        
 2   Category          506199 non-null  object        
 3   Category_id       506199 non-null  object        
 4   Subcategory       506199 non-null  object        
 5   Subcategory_id    506199 non-null  object        
 6   Project_Country   506199 non-null  object        
 7   Uid               506199 non-null  object        
 8   Launched          506199 non-null  datetime64[ns]
 9   Deadline          506199 non-null  datetime64[ns]
 10  Project_Currency  506199 non-null  object        
 11  Goal (USD)        506199 non-null  int64         
 12  Pledged (USD)     506199 non-null  int64         
 13  Backers (#)       496051 non-null  Int64         
 14  Stat

MOVE TO LATER!

When checking nulls we saw that there were nulls in the Backers (#) feature. Let's deal with those

In [19]:
# DON'T RUN CELL

#some more data cleaning

#fill nan with 0 in Backers (#) - already replaced empty spaces with na
data['Backers (#)'] = data['Backers (#)'].fillna(0) 
# change 

# check
data.info()

#change data type of Goal, Pledged, Backers (#) to int
#change_data_type(data, ['Goal (USD)', 'Pledged (USD)', 'Backers (#)'], int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506199 entries, 0 to 506198
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Case_ID           506199 non-null  object        
 1   Pid               506199 non-null  object        
 2   Category          506199 non-null  object        
 3   Category_id       506199 non-null  object        
 4   Subcategory       506199 non-null  object        
 5   Subcategory_id    506199 non-null  object        
 6   Project_Country   506199 non-null  object        
 7   Uid               506199 non-null  object        
 8   Launched          506199 non-null  datetime64[ns]
 9   Deadline          506199 non-null  datetime64[ns]
 10  Project_Currency  506199 non-null  object        
 11  Goal (USD)        506199 non-null  int64         
 12  Pledged (USD)     506199 non-null  int64         
 13  Backers (#)       506199 non-null  object        
 14  Stat

The data types look good

Let's check the Logic of the Status columns
- Successful campaigns should have more than 0 backers
- Campaigns where Pledged > Goal should be Successful
- Campaigns that were suspended or cancelled and with pledged < goal should not be Successful

In [96]:
#number of unique values 
data['Status'].nunique()
print(data['Status'].unique())

['Failed' 'Successful' 'Canceled']


Checks:
- Successful campaigns = Pledged > Goal
- Impute median values for backers respective to campaign status - canceled and suspended can be considered failed
- make sure classifications make sense 

In [70]:
# let's check campaigns where success label is incorrect

#check for Successful status where Pledged < Goal - These should be Failures
should_be_fails = data[(data['Status'] == 'Successful') & (data['Pledged (USD)'] < data['Goal (USD)'])]
should_be_fails

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status
63410,63411,1769772834,Film & Video,11,Shorts,32,United States of America,467587104,2015-11-12,2015-12-27,USD,36000,30716,275,Successful
96929,96930,780863434,Design,7,Product Design,28,European Union,1859408531,2016-02-08,2016-03-19,EUR,28174,28053,90,Successful
506024,506025,715247891,Comics,3,Comics,3,European Union,343048188,2020-03-12,2020-03-13,EUR,1,0,0,Successful


In [71]:
# Replace the values that match the condition with 'Failed'
data.loc[data['Pledged (USD)'] < data['Goal (USD)'], 'Status'] = 'Failed'

In [72]:
#check above
data[(data['Status'] == 'Successful') & (data['Pledged (USD)'] < data['Goal (USD)'])]

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status


Now let's check incorrect Failed

In [73]:
#check for Failed Campaigns where pledged > Goal - These should be successful 
should_be_success = (data['Status'] == 'Failed') & (data['Pledged (USD)'] > data['Goal (USD)'])

should_be_success.sum()

14

In [74]:
#replace above values 
data.loc[should_be_success, 'Status'] = 'Successful'

In [75]:
#check above
data[(data['Status'] == 'Failed') & (data['Pledged (USD)'] > data['Goal (USD)'])]

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status


Let's check the other status as well - in this case, corrections will become Failed or Successful as it is difficult to determine what causes a canceled and suspended campaign

In [76]:
#check for logic for Suspended and Cancelled
data[(data['Status'] == 'Suspended') | (data['Status'] == 'Canceled') & (data['Pledged (USD)'] > data['Goal (USD)'])]

# There are instances were the logic doesn't make sense (pledged > goal should be Successful), but the campaigns could have been suspended or cancelled. Let's see how many there are

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status
755,756,1211983964,Technology,16,Hardware,52,Japan,2008114165,2020-02-10,2020-03-15,JPY,29022,109956,83,Canceled
1880,1881,366287537,Fashion,9,Apparel,263,Japan,509340035,2018-06-29,2018-08-08,JPY,8984,9739,37,Canceled
2296,2297,2019198729,Technology,16,Hardware,52,Hong Kong,1185537397,2019-10-29,2019-12-28,HKD,128435,133133,39,Canceled
2393,2394,2119145473,Crafts,26,Stationery,353,Japan,731061123,2020-07-03,2020-08-10,JPY,9439,9565,32,Canceled
4623,4624,1747331098,Technology,16,Gadgets,337,Japan,1514234148,2019-08-31,2019-09-30,JPY,4632,15775,243,Suspended
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506165,506166,1739922754,Technology,16,Technology,16,Canada,420495623,2016-09-19,2016-10-12,CAD,1,11,15,Canceled
506166,506167,1836862932,Games,12,Video Games,35,Canada,373642459,2013-09-09,2013-10-14,CAD,1,16423,35,Canceled
506170,506171,1292523405,Fashion,9,Jewelry,267,United States of America,869720287,2019-08-22,2019-10-21,USD,1,61,3,Canceled
506175,506176,1315034272,Music,14,Hip-Hop,39,United States of America,670464031,2018-10-22,2018-11-22,USD,1,2,2,Canceled


For consistency, I will assume these are successful campaigns because I defined successful as the amount pledged > goal. So, let's change these.

In [79]:
# mask for more successes
also_successes = (data['Status'] == 'Suspended') | (data['Status'] == 'Canceled') & (data['Pledged (USD)'] > data['Goal (USD)'])

#replace values
data.loc[also_successes, 'Status'] = 'Successful'

In [80]:
#check above
data[(data['Status'] == 'Suspended') | (data['Status'] == 'Canceled') & (data['Pledged (USD)'] > data['Goal (USD)'])]

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status


okay, classifications make better sense now. let's move onto dealing with the nulls in backers (#)

In [82]:
data.isna().sum()

Case_ID                 0
Pid                     0
Category                0
Category_id             0
Subcategory             0
Subcategory_id          0
Project_Country         0
Uid                     0
Launched                0
Deadline                0
Project_Currency        0
Goal (USD)              0
Pledged (USD)           0
Backers (#)         10148
Status                  0
dtype: int64

In [83]:
#check for successful campaigns that have  backers - 9890 instances will replace these with medians
success_no_backers = (data['Backers (#)'].isna()) & (data['Status'] == 'Successful')
success_no_backers.sum()

9976

In [84]:
#get median values for successful campaigns - omit those that don't equal 0
median_backers_success = data.loc[(data['Status'] == 'Successful') & (data['Backers (#)'].notna()), 'Backers (#)'].median()

# Replace the values that are 0 with the median value
data.loc[success_no_backers, 'Backers (#)'] = median_backers_success

In [86]:
#check to see if above worked - should return empty df
data[(data['Backers (#)'].isna()) & (data['Status'] == 'Successful')]

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status


In [87]:
#check if any of the numeric columns have values under 0

under_0 =  data[(data['Goal (USD)'] < 0) | (data['Pledged (USD)'] < 0) | (data['Backers (#)'] < 0)]

under_0

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status
14428,14429,1317300700,Design,7,Product Design,28,United States of America,924758631,2014-11-19,2015-01-01,USD,150000,0,-2,Failed
117146,117147,1474098850,Design,7,Product Design,28,United States of America,1521927591,2014-12-09,2015-01-13,USD,20000,0,-2,Failed


In [88]:
# median failed
median_failed = data.loc[(data['Status'] == 'Failed') & (data['Backers (#)'].notna()), 'Backers (#)'].median()


#replace negative in Backers(#) - they were classified as Failed, so 0 makes sense plus Pledged was 0.
data.loc[under_0.index, 'Backers (#)'] = median_failed

In [89]:
#check above
data[(data['Goal (USD)'] < 0) | (data['Pledged (USD)'] < 0) | (data['Backers (#)'] < 0)]

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status


In [91]:
data.isna().sum()

Case_ID               0
Pid                   0
Category              0
Category_id           0
Subcategory           0
Subcategory_id        0
Project_Country       0
Uid                   0
Launched              0
Deadline              0
Project_Currency      0
Goal (USD)            0
Pledged (USD)         0
Backers (#)         172
Status                0
dtype: int64

In [93]:
# need to check backers for fails and others too
failed_backers = data[(data['Status'] == 'Failed') & (data['Backers (#)'].isna())]

#replace nulls
data.loc[failed_backers.index, 'Backers (#)'] = median_failed


In [94]:
# check work
data[(data['Status'] == 'Failed') & (data['Backers (#)'].isna())]

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status


In [95]:
#number of instances per status
data['Status'].value_counts()

Status
Failed        309596
Successful    196526
Canceled          77
Name: count, dtype: int64

So, it appears 'Suspended' campagins no longer exist which is fine as the success definition is met and consistent. Will move on to making a database

# Database

Move into python file

## Formatting

In [59]:
data.columns

Index(['Case_ID', 'Pid', 'Category', 'Category_id', 'Subcategory',
       'Subcategory_id', 'Project_Country', 'Uid', 'Launched', 'Deadline',
       'Project_Currency', 'Goal (USD)', 'Pledged (USD)', 'Backers (#)',
       'Status'],
      dtype='object')

I am going to drop Pid and Uid as I am focused more on the campaign specifics and not users. I will also add a ID column for Countries

In [61]:
data['Project_Country'].value_counts()

United States of America    370515
United Kingdom               50111
European Union               32463
Canada                       22064
Australia                    11457
Mexico                        5005
Sweden                        2974
Hong Kong                     2636
New Zealand                   2105
Denmark                       1781
Singapore                     1769
Switzerland                   1403
Norway                        1058
Japan                          840
Poland                          18
Name: Project_Country, dtype: int64

In [26]:
#declare dictionary with columns
country_dict = {'United States of America': 1, 'United Kingdom': 2, 'European Union': 3,
                'Canada': 4, 'Australia': 5, 'Mexico': 6, 'Sweden': 7, 'Hong Kong': 8,
                'New Zealand': 9, 'Denmark': 10, 'Singapore': 11, 'Switzerland': 12, 
                'Norway': 13, 'Japan': 14, 'Poland': 15}



#Add ID columns
data['Country_ID'] = data['Project_Country'].map(country_dict)

#change dtype
data['Country_ID'] = data['Country_ID'].astype(int)

#check
data.head(5)

,Case_ID,Pid,Category,Category_id,Subcategory,Subcategory_id,Project_Country,Uid,Launched,Deadline,Project_Currency,Goal (USD),Pledged (USD),Backers (#),Status,Country_ID
0,1,2137925650,Film & Video,11,Science Fiction,301,United States of America,1076478145,2016-08-11,2016-10-10,USD,100000000,0,0,Failed,1
1,2,1501531085,Film & Video,11,Fantasy,296,United States of America,224946798,2019-12-19,2020-02-14,USD,100000000,85,4,Failed,1
2,3,953415668,Technology,16,Software,51,Mexico,1772203542,2017-03-01,2017-03-22,MXN,5219374,1,1,Failed,6
3,4,1371386304,Publishing,18,Publishing,18,United States of America,1373465389,2018-06-04,2018-07-05,USD,100000000,1,1,Failed,1
4,5,1720842777,Art,1,Illustration,22,Canada,1455666383,2015-05-01,2015-06-30,CAD,80610122,0,0,Failed,4


In [27]:
#drop Pid and Uid - carry no value
data = data.drop(['Pid', 'Uid'], axis=1)

## Connection

In [31]:
#import libraries
import sqlite3
from sqlite3 import Error

In [28]:
#function to create connection
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

#function to execute queries
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")


def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

DB design...kind of

*Project Table*:
    - Contains information about individual campaigns (projects).
        - Columns:
            - caseID: Unique identifier for each project. PRIMARY
            - Launched: Date when the project was launched.
            - Deadline: Date when the project’s funding deadline ends.
            - categoryid: Foreign key referencing the Category table. SECONDARY
            - subcategoryid: Foreign key referencing the Subcategory table. SECONDARY
            - countryid: Foreign key referencing the Country table. SECONDARY
            - Goal: Funding goal in the specified currency.
            - Pledged: Amount pledged by backers (in USD or specified currency).
            - backers: Number of backers.
            - Status: Project status (e.g., successful, failed, ongoing).

*Category Table*:
    - Contains unique project categories.
        - Columns:
            - categoryid: Unique identifier for each category. PRIMARY
            - category: Name of the category (e.g., technology, art, music).

*Subcategory Table*:
    - Contains unique project subcategories.
        - Columns:
            - subcategoryid: Unique identifier for each subcategory. PRIMARY
            - subcategory: Name of the subcategory (e.g., mobile apps, painting, rock music).
            
*Country Table*:
    - Contains information about countries.
        - Columns:
            - countryid: Unique identifier for each country. PRIMARY
            - Country: Name of the country (e.g., United States, Canada).
            - Currency: Currency used in that country (e.g., USD, CAD).

In [39]:
#rename columns to match Tables
data.rename(columns={'Case_ID': 'Campaign_ID', 'Category_id': 'Category_ID',
                     'Subcategory_id': 'Subcategory_ID', 'Launched': 'Launch_Date',
                     'Deadline': 'End_Date', 'Goal (USD)': 'Goal',
                     'Pledged (USD)': 'Pledged', 'Backers (#)': 'Num_Donors'}, inplace=True)

#drop project_currency because it is redundant and Goal and Pledged are in USD
data = data.drop('Project_Currency', axis=1)

In [35]:
#create sqlite db and connect to it
connection = create_connection('kickstarter.sqlite')

Connection to SQLite DB successful


In [40]:
#define strings to create Tables

campaign_table = """
CREATE TABLE IF NOT EXISTS Campaigns (
    Campaign_ID INTEGER PRIMARY KEY,
    Launch_Date DATE,
    End_Date DATE,
    Category_ID INTEGER,
    Subcategory_ID INTEGER,
    Country_ID INTEGER,
    Goal INTEGER,
    Pledged INTEGER,
    Num_Donors INTEGER,
    Status TEXT,
    FOREIGN KEY (Category_ID) REFERENCES Categories,
    FOREIGN KEY (Subcategory_ID) REFERENCES Subcategories,
    FOREIGN KEY (Country_ID) REFERENCES Countries
);
"""

categories_table = """
CREATE TABLE IF NOT EXISTS Categories (
    Category_ID INTEGER PRIMARY KEY,
    Category TEXT
);
"""

subcategories_table = """
CREATE TABLE IF NOT EXISTS Subcategories (
    Subcategory_ID INTEGER PRIMARY KEY,
    Subcategory TEXT
);
"""

countries_table = """
CREATE TABLE IF NOT EXISTS Countries (
    Country_ID INTEGER PRIMARY KEY,
    Project_Country TEXT
);
"""

In [41]:
#create tables

#campaign table
execute_query(connection, campaign_table)

#categories table
execute_query(connection, categories_table)

#subcategories table
execute_query(connection, subcategories_table)

#countries table
execute_query(connection, countries_table)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [60]:
#declare table specific df to put into tables

#campaign_df
campaigns_df = data[['Campaign_ID', 'Launch_Date', 'End_Date', 'Category_ID',
                     'Subcategory_ID', 'Country_ID', 'Goal', 'Pledged',
                     'Num_Donors', 'Status']]

#category_df
category_df = data[['Category_ID', 'Category']]
#drop duplicates
category_df = category_df.drop_duplicates()


#subcategory_df
subcat_df = data[['Subcategory_ID', 'Subcategory']]
#drop duplicates
subcat_df = subcat_df.drop_duplicates()

#country_df
country_df = data[['Country_ID', 'Project_Country']]
#drop duplicates
country_df = country_df.drop_duplicates()


In [61]:
#put data into db tables using .to_sql

#campaign data
campaigns_df.to_sql('Campaigns', connection, if_exists='replace', index=False)

#categories data
category_df.to_sql('Categories', connection, if_exists='replace', index=False)

#subcategories data
subcat_df.to_sql('Subcategories', connection, if_exists='replace', index=False)

#countries data
country_df.to_sql('Countries', connection, if_exists='replace', index=False)


15

In [47]:
#function to check tables
def check_tables(table_name=str):
    query = f"SELECT * FROM {table_name} LIMIT 10"
    my_table = execute_read_query(connection, query)
    for item in my_table:
        print(item)

In [48]:
#test campaigns
check_tables('Campaigns')

('1', '2016-08-11 00:00:00', '2016-10-10 00:00:00', '11', '301', 1, 100000000, 0, 0, 'Failed')
('2', '2019-12-19 00:00:00', '2020-02-14 00:00:00', '11', '296', 1, 100000000, 85, 4, 'Failed')
('3', '2017-03-01 00:00:00', '2017-03-22 00:00:00', '16', '51', 6, 5219374, 1, 1, 'Failed')
('4', '2018-06-04 00:00:00', '2018-07-05 00:00:00', '18', '18', 1, 100000000, 1, 1, 'Failed')
('5', '2015-05-01 00:00:00', '2015-06-30 00:00:00', '1', '22', 4, 80610122, 0, 0, 'Failed')
('6', '2016-11-22 00:00:00', '2017-01-21 00:00:00', '13', '360', 1, 100000000, 1, 1, 'Failed')
('7', '2015-06-02 00:00:00', '2015-08-01 00:00:00', '16', '16', 1, 100000000, 0, 0, 'Failed')
('8', '2014-06-05 00:00:00', '2014-08-04 00:00:00', '16', '342', 1, 100000000, 56, 6, 'Failed')
('9', '2020-09-09 00:00:00', '2020-11-08 00:00:00', '18', '323', 1, 100000000, 4, 4, 'Failed')
('10', '2017-08-14 00:00:00', '2017-09-13 00:00:00', '11', '11', 1, 100000000, 1, 1, 'Failed')


In [49]:
#test categories
check_tables('Categories')

('11', 'Film & Video')
('11', 'Film & Video')
('16', 'Technology')
('18', 'Publishing')
('1', 'Art')
('13', 'Journalism')
('16', 'Technology')
('16', 'Technology')
('18', 'Publishing')
('11', 'Film & Video')


In [50]:
#test subcategories
check_tables('Subcategories')

('301', 'Science Fiction')
('296', 'Fantasy')
('51', 'Software')
('18', 'Publishing')
('22', 'Illustration')
('360', 'Video')
('16', 'Technology')
('342', 'Web')
('323', 'Academic')
('11', 'Film & Video')


In [51]:
#test countries
check_tables('Countries')

(1, 'United States of America')
(1, 'United States of America')
(6, 'Mexico')
(1, 'United States of America')
(4, 'Canada')
(1, 'United States of America')
(1, 'United States of America')
(1, 'United States of America')
(1, 'United States of America')
(1, 'United States of America')


In [66]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor = connection.cursor()
cursor.execute(sql_query)

table_names = cursor.fetchall()
print("List of tables:")
for table in table_names:
    print(table[0])

List of tables:
Campaigns
Categories
Subcategories
Countries


In [62]:
#check number of records in campaigns
cursor = connection.cursor()

# Count records in the Campaigns table
cursor.execute("SELECT COUNT(*) FROM Campaigns")
campaigns_count = cursor.fetchone()[0]
campaigns_count

506199

In [63]:
cursor = connection.cursor()

# Count records in the Campaigns table
cursor.execute("SELECT COUNT(*) FROM Categories")
cat_count = cursor.fetchone()[0]
cat_count

15

In [64]:
cursor = connection.cursor()

# Count records in the Campaigns table
cursor.execute("SELECT COUNT(*) FROM Subcategories")
subcat_count = cursor.fetchone()[0]
subcat_count

171

In [65]:
cursor = connection.cursor()

# Count records in the Campaigns table
cursor.execute("SELECT COUNT(*) FROM Countries")
countries_count = cursor.fetchone()[0]
countries_count

15